In [1]:
%reset -sf

import sys
sys.path.insert(0, '../qe')
import os
import glob
import shutil
import gc

import pandas as pd
import numpy as np
import scipy as sp

import get_data as gd
import atlas_plot as ap
plot = ap.Plotter()

import ROOT
from ROOT import TLorentzVector, TVector3

from matplotlib import pyplot as plt

Welcome to JupyROOT 6.28/04


In [2]:
branches = [
			"reco_e", 
			"reco_m", 
			"reco_leadlep", 
			"reco_subleadlep", 
			"reco_met", 
			"truth_e", 
			"truth_m", 
			"truth_leadlep", 
			"truth_subleadlep", 
			"truth_w", 
			"truth_h", 
			"truth_leadw", 
			"truth_subleadw", 
			"truth_met", 
			"truth_nu", 
			"truth_Bij"
			]

data = {}
for branch in branches:
	data[branch] = pd.read_hdf("/root/data/recotruth/full_345324_data.h5", key=branch)

In [7]:
data['truth_e']

,TruthRecoElPx,TruthRecoElPy,TruthRecoElPz,TruthRecoElE,TruthRecoElPt,TruthRecoElEta,TruthRecoElPhi,TruthRecoElM
0,-15869.748871,19304.493534,-40084.155512,47236.129426,24990.246094,-1.251097,2.258854,0.511
1,-517.080553,13331.143630,24005.032422,27463.218030,13341.167969,1.350111,1.609564,0.511
2,12629.886218,2484.921686,-12299.336730,17803.441969,12872.018555,-0.849562,0.194268,0.511
3,31907.491468,-9293.517188,-13961.017803,36046.740378,33233.378906,-0.408623,-0.283423,0.511
4,20694.524715,-11001.762173,-26413.084302,35312.223758,23437.195312,-0.968370,-0.488628,0.511
...,...,...,...,...,...,...,...,...
358010,23692.920533,-13394.202175,-13774.370227,30503.973687,27216.890625,-0.486658,-0.514533,0.511
358011,14515.394696,-3951.641235,-3464.120109,15437.366356,15043.674805,-0.228283,-0.265797,0.511
358012,16134.603276,-1244.076101,19589.589481,25409.155132,16182.495117,1.022704,-0.076954,0.511
358013,18211.642802,-1763.555733,5525.628417,19112.996417,18296.832031,0.297587,-0.096536,0.511


In [3]:
# Ensure ROOT is properly initialized
ROOT.gROOT.SetBatch(True)

# Function to calculate the CGLMP Bell inequality
def cglmp(z_xp, z_xm, z_yp, z_ym):
    tr_a = (8 / np.sqrt(3)) * (z_xp * z_xm + z_yp * z_ym)
    tr_b = 25 * (np.power(z_xp, 2) - np.power(z_yp, 2)) * (np.power(z_xm, 2) - np.power(z_ym, 2))
    tr_c = 100 * (z_xp * z_yp * z_xm * z_ym)
    tr = tr_a + tr_b + tr_c
    return tr

# Function to perform the boost operation using numpy and ROOT
def vectorize_boost(Higgs, particles):
    boosted_particles = np.zeros_like(particles)
    Higgs_boost = Higgs[:, :3] / Higgs[:, 3:4]

    for i in range(particles.shape[0]):
        p = ROOT.TLorentzVector(particles[i, 0], particles[i, 1], particles[i, 2], particles[i, 3])
        boost_vector = ROOT.TVector3(Higgs_boost[i, 0], Higgs_boost[i, 1], Higgs_boost[i, 2])
        p.Boost(-boost_vector)

        boosted_particles[i, 0] = p.Px()
        boosted_particles[i, 1] = p.Py()
        boosted_particles[i, 2] = p.Pz()
        boosted_particles[i, 3] = p.E()

    return boosted_particles

# Function to calculate Bij using ROOT's TLorentzVector and numpy
def calculate_Bij(particles):
    num_particles = particles.shape[0]

    # Extract particles
    Higgs = particles[:, :4]
    WpBoson = particles[:, 4:8]
    WpLepton = particles[:, 8:12]
    WnBoson = particles[:, 16:20]
    WnLepton = particles[:, 20:24]

    # Boost particles to the Higgs rest frame
    boosted_WpBoson = vectorize_boost(Higgs, WpBoson)
    boosted_WpLepton = vectorize_boost(Higgs, WpLepton)
    boosted_WnBoson = vectorize_boost(Higgs, WnBoson)
    boosted_WnLepton = vectorize_boost(Higgs, WnLepton)

    # Define orthogonal basis vectors (k, r, n)
    k = boosted_WpBoson[:, :3] / np.linalg.norm(boosted_WpBoson[:, :3], axis=1, keepdims=True)
    p = np.array([0, 0, 1])

    y = np.dot(k, p)
    r_length = np.sqrt(1 - np.square(y))
    r = (1 / r_length[:, np.newaxis]) * (p - y[:, np.newaxis] * k)
    n = (1 / r_length[:, np.newaxis]) * np.cross(p, k)

    # Project onto (k, r, n) frame
    def project_onto_basis(vectors, n, r, k):
        return np.array([np.einsum('ij,ij->i', vectors[:, :3], n), 
                         np.einsum('ij,ij->i', vectors[:, :3], r), 
                         np.einsum('ij,ij->i', vectors[:, :3], k), 
                         vectors[:, 3]]).T

    WpLp_k = project_onto_basis(boosted_WpLepton, n, r, k)
    WnLp_k = project_onto_basis(boosted_WnLepton, n, r, k)

    # Compute directional cosines
    cos_n_join_p = WpLp_k[:, 0] / np.linalg.norm(WpLp_k[:, :3], axis=1)
    cos_r_join_p = WpLp_k[:, 1] / np.linalg.norm(WpLp_k[:, :3], axis=1)
    cos_k_join_p = WpLp_k[:, 2] / np.linalg.norm(WpLp_k[:, :3], axis=1)

    cos_n_join_n = WnLp_k[:, 0] / np.linalg.norm(WnLp_k[:, :3], axis=1)
    cos_r_join_n = WnLp_k[:, 1] / np.linalg.norm(WnLp_k[:, :3], axis=1)
    cos_k_join_n = WnLp_k[:, 2] / np.linalg.norm(WnLp_k[:, :3], axis=1)

    # Calculate Bell inequalities
    B_xy = cglmp(cos_n_join_p, cos_n_join_n, cos_r_join_p, cos_r_join_n)
    B_yz = cglmp(cos_r_join_p, cos_r_join_n, cos_k_join_p, cos_k_join_n)
    B_zx = cglmp(cos_n_join_p, cos_n_join_n, cos_k_join_p, cos_k_join_n)

    return np.stack([B_xy, B_yz, B_zx], axis=1)

In [4]:
lead = data["truth_leadlep"][["TruthRecoLeadLepPx", "TruthRecoLeadLepPy", "TruthRecoLeadLepPz", "TruthRecoLeadLepE"]] / 1e3
sublead = data["truth_subleadlep"][["TruthRecoSubleadLepPx", "TruthRecoSubleadLepPy", "TruthRecoSubleadLepPz", "TruthRecoSubleadLepE"]] / 1e3
w_lead = data["truth_leadw"][["TruthRecoLeadWPx", "TruthRecoLeadWPy", "TruthRecoLeadWPz", "TruthRecoLeadWE"]] / 1e3
w_sublead = data["truth_subleadw"][["TruthRecoSubleadWPx", "TruthRecoSubleadWPy", "TruthRecoSubleadWPz", "TruthRecoSubleadWE"]] / 1e3
h = w_lead + w_sublead
Bij = data["truth_Bij"]
# particles = np.concatenate([h.to_numpy(), w_lead.to_numpy(), lead.to_numpy(), w_sublead.to_numpy(), sublead.to_numpy()], axis=1)

In [32]:
calculate_Bij(particles)

array([[nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan],
       ...,
       [nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan]])